In [29]:
import nbimporter
import functions
reload(functions)
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
import numpy as np
import itertools
#viz
from datetime import date
from random import randint
import math

from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_file, show
from bokeh.models import FuncTickFormatter
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.layouts import column, row
from bokeh.layouts import gridplot
from bokeh.plotting import figure

###################



#from bokeh.io import vform



###################
from sklearn import ensemble
from sklearn.metrics.pairwise import linear_kernel
from dateutil import parser

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
from matplotlib import pyplot

#from bokeh.io import vform

Importing Jupyter notebook from functions.ipynb


In [2]:
import sys
sys.version
sys.version_info

sys.version_info(major=2, minor=7, micro=13, releaselevel='final', serial=0)

In [3]:
import splunklib.client as client
import httplib
import urllib
from xml.etree import ElementTree
import ssl

In [4]:
from splunklib.binding import HTTPError
import splunklib.client as client


In [5]:
# HOST = "**"
# PORT = 8089
# USERNAME = "**"
# PASSWORD = "**"

# # Present credentials to Splunk and retrieve the session key
# connection = httplib.HTTPSConnection(HOST, PORT,context=ssl._create_unverified_context())
# body = urllib.urlencode({'username': USERNAME, 'password': PASSWORD})
# headers = { 
#     'Content-Type': "application/x-www-form-urlencoded", 
#     'Content-Length': str(len(body)),
#     'Host': HOST,
#     'User-Agent': "a.py/1.0",
#     'Accept': "*/*"
# }

# print 'connecting'

# try:
#     connection.request("POST", "/services/auth/login", body, headers)
#     print 'here'
#     response = connection.getresponse()
# finally:
#     print 'closing'
#     connection.close()
#     print 'closed'
# if response.status != 200:
#     raise Exception, "%d (%s)" % (response.status, response.reason)
# print 'reading'
# body = response.read()
# print 'read'
# #sessionKey = ElementTree.XML(body).findtext("./sessionKey")


In [6]:
# import splunklib.results as results
# import splunklib.client as client

# service = client.Service(host=HOST, port=PORT, username=USERNAME, password=PASSWORD)
# service.login()
# # Run a one-shot search and display the results using the results reader

# # Set the parameters for the search:
# # - Search everything in a 24-hour time range starting June 19, 12:00pm
# # - Display the first 10 results
# kwargs_oneshot = {"earliest_time": "2017-06-19T12:00:00.000-07:00",
#                   "latest_time": "2017-06-19T12:02:00.000-07:00"}
# searchquery_oneshot = "search * | head 10"

# oneshotsearch_results = service.jobs.oneshot(searchquery_oneshot, **kwargs_oneshot)

# # Get the results and display them using the ResultsReader
# reader = results.ResultsReader(oneshotsearch_results)
# for item in reader:
#     print(item)

In [8]:
#functions.test()

In [9]:
# new_file = 'new_logs'

# df_new = pd.read_csv(new_file)
# df_new.head()

In [174]:
log_file = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file1 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_7.csv'
log_file2 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_1.csv'
log_file3 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file4 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'

# log_file = '/Users/momori/Documents/projects/log_monitoring/11_6_errors.csv'
# log_file1 = '/Users/momori/Documents/projects/log_monitoring/10_30_errors.csv'
# log_file2 = '/Users/momori/Documents/projects/log_monitoring/10_23_errors.csv'
# log_file3 = '/Users/momori/Documents/projects/log_monitoring/10_16_errors.csv'



df_test = pd.read_csv(log_file)
df_train1 = pd.read_csv(log_file1)
df_train2 = pd.read_csv(log_file2)
df_train3 = pd.read_csv(log_file2)

In [175]:
train_df = [df_train1, df_train2, df_train3]
#train_df = [df_new]

## Functions

In [189]:
##km: clustering model
##vecetorizer: document vectorizer
##document: document to cluster
def predict_document_cluster(km, vectorizer, document):
    doc = vectorizer.transform([document])
    return km.predict(doc)

def format_df(df):
    r_df = df.dropna(axis=1)
    r_df.loc[:,'_time'] = r_df['_time'].apply(lambda x: format_time(x))
    return r_df

def format_time(string):
    ##may need smarter logic. seems slow
    return parser.parse(string)

##step_time in minutes
##
## returns: dict. key is timestamp, value is dataframe
def split_by_time(df, step_time=30):
    r_dict = {}
    t=str(step_time)+'Min'
    
    #set _time to index
    tmp = df.set_index(1)
    tmp = tmp.groupby(pd.TimeGrouper(freq=t))
    for key, item in tmp:
        try:
            r_dict[str(key)] = tmp.get_group(key)
        except:
            r_dict[str(key)] = None
    return r_dict
        
def show_top_words(km, vect, show=False, save=True, out_filename='output'):
    dcluster_words = {}
    if show:
        print("Top terms per cluster:")

    order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vect.get_feature_names()
    for i in range(k):
        if show:
            print("Cluster %d:" % i)
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind])
        if save:
            words = []
            for ind in order_centroids[i, :10]:
                words.append(terms[ind])
            dcluster_words[i] = words
    if save:
        np.save(out_filename, dcluster_words)
    return dcluster_words

def cluster_counts(df):
    return df.groupby(2).size()
    
    


In [190]:
# d = np.load('output.npy').item()
# for k,v1 in d.iteritems():
#     print k,v1

In [191]:
# llogs_train = []
# llogs_train.append(df_new['_raw'].tolist())

In [192]:
len(train_df)

3

In [193]:
##variables
look_back = 1
llogs_train = []
for i in range(look_back):
    llogs_train.append(train_df[i]['_raw'].tolist())
logs_test = df_test['_raw'].tolist()


#logs_train1 = df_train1['_raw'].tolist()
#logs_train2 = df_train2['_raw'].tolist()
#logs_test = df_test['_raw'].tolist()

In [194]:
len(llogs_train)

1

In [195]:
##Fit a vectorizer
vect = TfidfVectorizer(max_df = 0.5, min_df = 2, stop_words='english')

#v is base vectorizer, can be used to transform other logs.
ve = vect.fit(list(itertools.chain(*llogs_train)))

#x is the tfidf sparse matrix created for this set of logs
l_vectors = []
for i in range(min(look_back, len(llogs_train))):
    l_vectors.append(ve.transform(llogs_train[i]))
#x1 = v.transform(logs_train1)
#x2 = v.transform(logs_train2)

#make a matrix using all data
input_matrix = ve.transform(list(itertools.chain(*llogs_train)))


In [196]:
##create kmeans model and fit
k=20
km = MiniBatchKMeans(n_clusters=k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000)
km.fit(input_matrix)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=1000, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [197]:
##Let's use cluster 6 as it shows '500' as one of the top words
cluster_words = show_top_words(km, vect, show=True, save=True)

Top terms per cluster:
Cluster 0:
 mule
 loggermessageprocessor
 processor
 default
 api
 connector
 enterpriseaccount
 http
 receiver
 accountservice
Cluster 1:
 gst
 address
 globalsalestax
 null
 status
 failure
 parameterizedmessagefactory
 log4j
 59e02f1e
 7a7f3a1f
Cluster 2:
 customeraccountnumber
 provided
 invalid
 errors
 moreinfo
 type
 202106
 ent
 entitledofferings
 calling
Cluster 3:
 set
 falloutord0_
 falloutord2_
 transaction_id
 falloutacc0_
 falloutadd0_
 falloutoff0_
 create_date
 update_date
 fallouttax0_
Cluster 4:
 rps
 restrictions
 contactinformation
 en_us
 addresstype
 fe80
 custommap
 webserver
 ebcsphase
 intuit_ebcs_assetalias
Cluster 5:
 java
 mule
 execute
 execution
 blockingprocessorexecutor
 processor
 exceptiontomessagingexceptionexecutioninterceptor
 net
 messageprocessorexecutiontemplate
 chain
Cluster 6:
 entitlement
 entitledoffering
 entitlementofferingrestransformer
 received
 transformer
 connectorwithoutmulesession
 cas
 receiver
 06
 12
Clust

In [198]:
#for i in range(20):
#    print i, len(df_predict1[df_predict1[2]==i]), len(df_predict2[df_predict2[2]==i])

In [199]:
import pickle
with open('cluster_words_2', 'wb') as fp:
    pickle.dump(cluster_words, fp)

In [200]:
#goal: create df with columns [logs, times, labels] for each 
l_formatted_df = []
for i in range(min(look_back, len(llogs_train))):
    formatted_df = format_df(train_df[i])
    times = formatted_df['_time'].tolist()
    mat = ve.transform(llogs_train[i])
    labels = km.predict(mat)
    lst = zip(llogs_train[i], times, labels.tolist())
    tmp_df = pd.DataFrame(lst)
    l_formatted_df.append(tmp_df)
    

In [201]:
l_formatted_df[0].to_csv('new_df_formatted_2')

# Need to add some logic so if no results in a given 30 minute interval, will put a 0 count

maybe done with the try/except clause change

In [202]:
#split by cluster number
d_cluster_to_od = {}
for cluster_num in range(k):
    l_df = []
    for i in range(look_back):
        filtered_df = l_formatted_df[i][l_formatted_df[i][2]==cluster_num]
        l_df.append(OrderedDict(sorted(split_by_time(filtered_df).items())))
    d_cluster_to_od[cluster_num] = l_df

In [203]:
##from the ordered dict, get the counts
##d_counts is a dictionary of dictionary. {cluster_num: {timestamp: [count]}}
d_counts = {}
for cluster_num in range(k):
    for odict in d_cluster_to_od[cluster_num]:
        for timestamp, df in odict.iteritems():
            inner_dict = OrderedDict()
            try: 
                count = df.shape[0]
            except:
                count = 0
            if cluster_num not in d_counts.keys():
                #new entry in d_counts. Create inner dict
                inner_dict[timestamp] = [count]
                d_counts[cluster_num] = inner_dict
            elif timestamp not in d_counts[cluster_num].keys():
                #inner dict is there. add new key/value pair
                d_counts[cluster_num][timestamp] = [count]
            else:
                #cluster num is there and timestamp is there. then we append
                d_counts[cluster_num][timestamp].append(count)

            

In [204]:
##now for each cluster_num, can use ARIMA model to predict.
times = []
for cluster_num in range(k):
    d_timestamp_counts = d_counts[cluster_num]
    for time, counts in d_timestamp_counts.iteritems():
        times.append(time)
        print time, counts

2017-08-07 00:00:00-07:00 [1]
2017-08-07 00:30:00-07:00 [0]
2017-08-07 01:00:00-07:00 [1]
2017-08-07 01:30:00-07:00 [0]
2017-08-07 02:00:00-07:00 [3]
2017-08-07 02:30:00-07:00 [2]
2017-08-07 03:00:00-07:00 [73]
2017-08-07 03:30:00-07:00 [6]
2017-08-07 04:00:00-07:00 [33]
2017-08-07 04:30:00-07:00 [14]
2017-08-07 05:00:00-07:00 [11]
2017-08-07 05:30:00-07:00 [11]
2017-08-07 06:00:00-07:00 [8]
2017-08-07 06:30:00-07:00 [33]
2017-08-07 07:00:00-07:00 [31]
2017-08-07 07:30:00-07:00 [54]
2017-08-07 08:00:00-07:00 [48]
2017-08-07 08:30:00-07:00 [54]
2017-08-07 09:00:00-07:00 [75]
2017-08-07 09:30:00-07:00 [94]
2017-08-07 10:00:00-07:00 [79]
2017-08-07 10:30:00-07:00 [83]
2017-08-07 11:00:00-07:00 [61]
2017-08-07 11:30:00-07:00 [72]
2017-08-07 12:00:00-07:00 [103]
2017-08-07 12:30:00-07:00 [76]
2017-08-07 13:00:00-07:00 [84]
2017-08-07 13:30:00-07:00 [62]
2017-08-07 14:00:00-07:00 [53]
2017-08-07 14:30:00-07:00 [46]
2017-08-07 15:00:00-07:00 [44]
2017-08-07 15:30:00-07:00 [56]
2017-08-07 16:0

2017-08-08 08:30:00-07:00 [0]
2017-08-08 09:00:00-07:00 [0]
2017-08-08 09:30:00-07:00 [0]
2017-08-08 10:00:00-07:00 [0]
2017-08-08 10:30:00-07:00 [0]
2017-08-08 11:00:00-07:00 [0]
2017-08-08 11:30:00-07:00 [0]
2017-08-08 12:00:00-07:00 [0]
2017-08-08 12:30:00-07:00 [0]
2017-08-08 13:00:00-07:00 [0]
2017-08-08 13:30:00-07:00 [0]
2017-08-08 14:00:00-07:00 [0]
2017-08-08 14:30:00-07:00 [0]
2017-08-08 15:00:00-07:00 [0]
2017-08-08 15:30:00-07:00 [0]
2017-08-08 16:00:00-07:00 [0]
2017-08-08 16:30:00-07:00 [0]
2017-08-08 17:00:00-07:00 [0]
2017-08-08 17:30:00-07:00 [0]
2017-08-08 18:00:00-07:00 [0]
2017-08-08 18:30:00-07:00 [0]
2017-08-08 19:00:00-07:00 [0]
2017-08-08 19:30:00-07:00 [0]
2017-08-08 20:00:00-07:00 [0]
2017-08-08 20:30:00-07:00 [0]
2017-08-08 21:00:00-07:00 [55]
2017-08-07 06:30:00-07:00 [1]
2017-08-07 07:00:00-07:00 [4]
2017-08-07 07:30:00-07:00 [0]
2017-08-07 08:00:00-07:00 [0]
2017-08-07 08:30:00-07:00 [41]
2017-08-07 09:00:00-07:00 [0]
2017-08-07 09:30:00-07:00 [1]
2017-08-

In [205]:
import pickle 

with open('times_2', 'wb') as fp:
    pickle.dump(times, fp)

In [206]:
# #for each cluster, make a dictionary of counts. eg {0:[100,150,125]}
# d_counts = {}
# for cluster_num in range(k):
#     l_cluster_df = d_cluster_to_df[cluster_num]
#     lst = []
#     for df in l_cluster_df:
#         print df.shape[0]
        



In [207]:
print k

20


In [208]:
# ##get predictions for a subset of the logs and create df with _raw, _time, and label
formatted_df = format_df(df_train1)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = ve.transform(logs)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
df_predict1 = pd.DataFrame(lst)
df_predict1.head()

##do same for second train set
formatted_df = format_df(df_train2)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x2 = ve.transform(logs)
labels = km.predict(test_x2)
lst = zip(logs, times, labels.tolist())
df_predict2 = pd.DataFrame(lst)
df_predict2.head()

formatted_df = format_df(df_train3)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x3 = ve.transform(logs)
labels = km.predict(test_x3)
lst = zip(logs, times, labels.tolist())
df_predict3 = pd.DataFrame(lst)
df_predict3.head()

,0,1,2
0,"ERROR 2017-08-02 23:58:04,878 [[commerce-enter...",2017-08-02 23:58:04.878000-07:00,0
1,"ERROR 2017-08-02 23:58:04,865 [[commerce-enter...",2017-08-02 23:58:04.865000-07:00,14
2,Error Not Set\nError Not Set,2017-08-02 23:57:28-07:00,3
3,Error Not Set\nError Not Set,2017-08-02 23:57:24-07:00,3
4,Error Not Set\nError Not Set,2017-08-02 23:57:22-07:00,3


In [209]:
df_predict1.shape

(40557, 3)

In [210]:
# ##for each cluster number, create a dt
# df_clusters1, df_clusters2 = None


In [211]:
cluster_words.keys()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [212]:
print zip(cluster_words.keys(),cluster_counts(df_predict1), cluster_counts(df_predict2), cluster_counts(df_predict3))

[(0, 3039, 19674, 19674), (1, 3550, 5, 5), (2, 2801, 2417, 2417), (3, 7711, 9783, 9783), (4, 501, 534, 534), (5, 767, 408152, 408152), (6, 1045, 1146, 1146), (7, 3638, 7, 7), (8, 467, 495, 495), (9, 1310, 11178, 11178), (10, 150, 10987, 10987), (11, 95, 37, 37), (12, 1127, 55507, 55507), (13, 6171, 9, 9), (14, 1512, 1491, 1491), (15, 1745, 112757, 112757), (16, 441, 5297, 5297), (17, 3512, 10, 10), (18, 98, 87, 87), (19, 877, 1, 1)]


In [213]:
df_predict1[df_predict3[2]==15]

/Users/momori/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,0,1,2
30,"WARN 2017-08-08 23:29:17,952 [ActiveMQ Sessio...",2017-08-08 23:29:17.952000-07:00,5
31,"ERROR 2017-08-08 23:27:02,574 [[commerce-entit...",2017-08-08 23:27:02.574000-07:00,2
32,"ERROR 2017-08-08 23:22:26,695 [[commerce-enter...",2017-08-08 23:22:26.695000-07:00,0
33,"ERROR 2017-08-08 23:22:26,681 [[commerce-enter...",2017-08-08 23:22:26.681000-07:00,14
4321,intuit_webs_transaction_datetime=2017-08-0...,2017-08-08 08:34:50-07:00,12
4323,"ERROR 2017-08-08 08:34:46,518 [[commerce-entit...",2017-08-08 08:34:46.518000-07:00,2
4348,"ERROR 2017-08-08 08:31:39,101 [[commerce-entit...",2017-08-08 08:31:39.101000-07:00,2
4349,"ERROR 2017-08-08 08:31:35,313 [[commerce-enter...",2017-08-08 08:31:35.313000-07:00,0
4350,"ERROR 2017-08-08 08:31:35,307 [[commerce-enter...",2017-08-08 08:31:35.307000-07:00,0
4351,"ERROR 2017-08-08 08:31:31,797 [[commerce-enter...",2017-08-08 08:31:31.797000-07:00,0


In [240]:
cluster_num = 2

np.save('cluster_num_2', cluster_num)

f_df1 = df_predict1[df_predict1[2]==cluster_num]
f_df2 = df_predict2[df_predict2[2]==cluster_num]
f_df3 = df_predict3[df_predict3[2]==cluster_num]
#f_df1 = df_predict1
#f_df2 = df_predict2
f_df3.head()

,0,1,2
21,"ERROR 2017-08-02 23:44:36,890 [[commerce-entit...",2017-08-02 23:44:36.890000-07:00,2
45,"ERROR 2017-08-02 23:10:59,131 [[commerce-entit...",2017-08-02 23:10:59.131000-07:00,2
68,"ERROR 2017-08-02 22:57:39,015 [[commerce-entit...",2017-08-02 22:57:39.015000-07:00,2
69,"ERROR 2017-08-02 22:56:04,095 [[commerce-entit...",2017-08-02 22:56:04.095000-07:00,2
90,"ERROR 2017-08-02 22:42:46,902 [[commerce-entit...",2017-08-02 22:42:46.902000-07:00,2


In [241]:
check_logs = f_df1[0].tolist()

In [242]:
set(check_logs)

{'ERROR 2017-08-07 06:12:15,142 [[commerce-entitlement-2.1.0].ConnectorWithoutMuleSession.receiver.2533] After calling CAS Get EntitledOfferings Response Payload :: Errors: <?xml version="1.0" encoding="UTF-8" standalone="yes"?><Errors xmlns="http://schema.intuit.com/platform/common/error/v2"><error><code>ENT-202106</code><type>CLIENT</type><message>CustomerAccountNumber provided is Invalid</message><detail>CustomerAccountNumber provided is Invalid</detail><moreInfo>CustomerAccountNumber provided is Invalid</moreInfo></error></Errors>',
 'ERROR 2017-08-07 07:04:33,286 [[commerce-entitlement-2.1.0].ConnectorWithoutMuleSession.receiver.2621] After calling CAS Get EntitledOfferings Response Payload :: Errors: <?xml version="1.0" encoding="UTF-8" standalone="yes"?><Errors xmlns="http://schema.intuit.com/platform/common/error/v2"><error><code>ENT-202106</code><type>CLIENT</type><message>CustomerAccountNumber provided is Invalid</message><detail>CustomerAccountNumber provided is Invalid</det

In [243]:
with open('check_logs_2', 'wb') as fp:
    pickle.dump(check_logs[:20], fp)

In [244]:
##Create dictionary of dataframes, split by time
d1 = OrderedDict(sorted(split_by_time(f_df1, step_time=30).items()))
d2 = OrderedDict(sorted(split_by_time(f_df2, step_time=30).items()))
d3 = OrderedDict(sorted(split_by_time(f_df3, step_time=30).items()))

#d1 = OrderedDict(sorted(split_by_time(df_predict1).items()))
#d2 = OrderedDict(sorted(split_by_time(df_predict2).items()))


In [245]:
d3

OrderedDict([('2017-08-01 00:00:00-07:00',
                                                                                                0  \
              1                                                                                     
              2017-08-01 00:18:44.377000-07:00  ERROR 2017-08-01 00:18:44,377 [[commerce-entit...   
              2017-08-01 00:21:33.788000-07:00  ERROR 2017-08-01 00:21:33,788 [[commerce-entit...   
              2017-08-01 00:27:19.521000-07:00  ERROR 2017-08-01 00:27:19,521 [[commerce-entit...   
              
                                                2  
              1                                    
              2017-08-01 00:18:44.377000-07:00  2  
              2017-08-01 00:21:33.788000-07:00  2  
              2017-08-01 00:27:19.521000-07:00  2  ),
             ('2017-08-01 00:30:00-07:00', None),
             ('2017-08-01 01:00:00-07:00',
                                                                                  

In [246]:
d3.iteritems()

<generator object iteritems at 0x2265d2d20>

In [247]:
#create counts for training
counts1 = []
counts2 = []
counts3=[]
for k, val in d1.iteritems():
    if d1[k] is not None:
        counts1.append(val.shape[0])
    else:
        counts1.append(0)

for k, val in d2.iteritems():
    if d2[k] is not None:
        counts2.append(val.shape[0])
    else:
        counts2.append(0)
        
for k, val in d3.iteritems():
    if d3[k] is not None:
        counts3.append(val.shape[0])
    else:
        counts3.append(0)

In [248]:
len(counts1), len(counts2), len(counts3)

(96, 96, 96)

In [249]:
np.save('counts_2', counts2)

In [250]:
counts1[:10], counts2[:10]
zip(counts1, counts2, counts3)

[(5, 3, 3),
 (3, 0, 0),
 (3, 2, 2),
 (3, 6, 6),
 (2, 6, 6),
 (4, 2, 2),
 (6, 7, 7),
 (8, 3, 3),
 (24, 28, 28),
 (28, 35, 35),
 (50, 48, 48),
 (47, 54, 54),
 (78, 73, 73),
 (67, 80, 80),
 (78, 73, 73),
 (83, 70, 70),
 (76, 61, 61),
 (87, 47, 47),
 (55, 58, 58),
 (49, 49, 49),
 (54, 41, 41),
 (66, 47, 47),
 (60, 45, 45),
 (63, 35, 35),
 (49, 46, 46),
 (49, 47, 47),
 (41, 30, 30),
 (47, 37, 37),
 (38, 23, 23),
 (45, 29, 29),
 (28, 24, 24),
 (16, 22, 22),
 (22, 12, 12),
 (16, 12, 12),
 (19, 9, 9),
 (10, 9, 9),
 (7, 5, 5),
 (8, 7, 7),
 (8, 4, 4),
 (11, 3, 3),
 (7, 1, 1),
 (4, 7, 7),
 (5, 3, 3),
 (3, 2, 2),
 (2, 7, 7),
 (2, 3, 3),
 (0, 1, 1),
 (1, 3, 3),
 (2, 3, 3),
 (0, 2, 2),
 (3, 2, 2),
 (4, 4, 4),
 (1, 3, 3),
 (2, 3, 3),
 (6, 3, 3),
 (6, 17, 17),
 (22, 15, 15),
 (38, 33, 33),
 (51, 41, 41),
 (55, 48, 48),
 (89, 65, 65),
 (82, 72, 72),
 (98, 69, 69),
 (60, 49, 49),
 (76, 59, 59),
 (73, 60, 60),
 (63, 58, 58),
 (61, 52, 52),
 (45, 56, 56),
 (46, 56, 56),
 (57, 39, 39),
 (49, 43, 43),
 (58,

In [251]:
# v = vect.fit(logs_train1)
# v.transform(logs_test)

In [252]:
##Let's fit the 'future' data and pull out cluster_num
formatted_df = format_df(df_test)
logs_test = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = ve.transform(logs_test)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
test_data = pd.DataFrame(lst)
test_data.head()

,0,1,2
0,"ERROR 2017-08-02 23:58:04,878 [[commerce-enter...",2017-08-19 14:18:11.732000-07:00,4
1,"ERROR 2017-08-02 23:58:04,865 [[commerce-enter...",2017-08-19 14:18:09.124000-07:00,12
2,Error Not Set\nError Not Set,2017-08-19 14:17:48.122000-07:00,12
3,Error Not Set\nError Not Set,2017-08-19 14:17:47.376000-07:00,12
4,Error Not Set\nError Not Set,2017-08-19 14:17:47.374000-07:00,12


In [253]:
test_data = test_data[test_data[2]==cluster_num]

#test_data = test_data
test_d = OrderedDict(sorted(split_by_time(test_data).items()))
test_counts = []
for k, val in test_d.iteritems():
    if val is None:
        test_counts.append(0)
    else:
        test_counts.append(val.shape[0])


In [254]:
# test_data.head()

In [255]:
# len(test_counts)

In [256]:
# test_counts[:10]

In [275]:
#zip the data
counts1 = [float(i) for i in counts1]
counts2 = [float(i) for i in counts2]
counts3 = [float(i) for i in counts3]

train = zip(counts1, counts2)#, counts3)
train = [i for i in train if math.fabs(i[1]-i[0] < 100)]
print len(train)

96


In [283]:
zip(train, pred_tuples)

[((5.0, 3.0), (2.0400360154599446, 5.9599639845400532)),
 ((3.0, 0.0), (-1.4399459768100817, 4.4399459768100806)),
 ((3.0, 2.0), (1.5200180066641709, 3.4799819912042249)),
 ((3.0, 6.0), (1.5600540231899171, 7.4399459768100797)),
 ((2.0, 6.0), (0.080072030919890569, 7.9199279690801063)),
 ((4.0, 2.0), (1.0400360154599451, 4.9599639845400532)),
 ((6.0, 7.0), (5.52001800666417, 7.4799819912042249)),
 ((8.0, 3.0), (0.60009003864986354, 10.399909961350133)),
 ((24.0, 28.0), (22.080072030919887, 29.919927969080099)),
 ((28.0, 35.0), (24.6401260541098, 38.359873945890179)),
 ((50.0, 48.0), (47.040036015459933, 50.959963984540039)),
 ((47.0, 54.0), (43.6401260541098, 57.359873945890172)),
 ((78.0, 73.0), (70.600090038649839, 80.399909961350104)),
 ((67.0, 80.0), (60.760234100489619, 86.239765899510317)),
 ((78.0, 73.0), (70.600090038649839, 80.399909961350104)),
 ((83.0, 70.0), (63.760234100489619, 89.239765899510317)),
 ((76.0, 61.0), (53.800270115949566, 83.199729884050384)),
 ((87.0, 47.0),

## ARIMA prediction

In [278]:
##make predictions by fitting ARIMA model into the lists
prediction_upperbound = []
prediction_lowerbound = []
prediction_value = []

for t in train:
    rgr = ARIMA(t, order=(0,0,0))
    fit = rgr.fit(disp=0)
    forecast =  fit.forecast()
    prediction_upperbound.append(forecast[2][0][1])
    prediction_lowerbound.append(forecast[2][0][0])
    prediction_value.append(forecast[0])

#Turn into numpy array for computation
prediction_upperbound = np.asarray(prediction_upperbound)
prediction_lowerbound = np.asarray(prediction_lowerbound)
prediction_value = np.asarray(prediction_value)

/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [279]:
def format_tuples(tuples):
    for tup in tuples:
        if tup[0] > tup[1]:
            tup[1] = temp
            tup[1] = tup[0]
            tup[0] = temp
    return tuples
        
def analyze_tuples(tuples):
    sum_diff = 0
    for tup in tuples:
        if tup[0] > tup[1]:
            print tup
        sum_diff+=tup[1]-tup[0]
    return sum_diff

In [280]:
analyze_tuples(pred_tuples)

1197.420904394919

In [281]:
pred_tuples=zip(prediction_lowerbound, prediction_upperbound)
#pred_tuples = format_tuples(pred_tuples)

In [282]:
np.save('upperbound_2', zip(*pred_tuples)[1])
np.save('lowerbound_2', zip(*pred_tuples)[0])


## Vis

In [ ]:

# # Define Bollinger Bands.
# upperband = prediction_upperbound
# lowerband = prediction_lowerbound
# x_data = np.arange(1, 101)

# length = min(len(prediction_upperbound), len(prediction_lowerbound))
# x_data = np.arange(1, length)
# # Bollinger shading glyph:
# band_x = np.append(x_data, x_data[::-1])
# band_y = np.append(lowerband, upperband[::-1])



# p = figure(x_axis_type='datetime', title="")

# # add a line renderer
# p.line([i for i in range(1,length)], counts2, line_width=2)
# p.grid.grid_line_alpha = 0.4
# p.x_range.range_padding = 0
# p.plot_height = 600
# p.plot_width = 800

# p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)



# label_dict = {}
# for i, s in enumerate(times):
#     label_dict[i] = s

# p.xaxis.formatter = FuncTickFormatter(code="""
#     var labels = %s;
#     return labels[tick];
# """ % label_dict)

# output_file('bollinger.html', title='Bollinger bands (file)')


# # data = dict(
# #         dates=[date(2014, 3, i+1) for i in range(10)],
# #         downloads=[randint(0, 100) for i in range(10)],
# #     )


# text_df = pd.DataFrame(check_logs[:4]).reset_index()
# text_df.columns=['index','text']
# source = ColumnDataSource(text_df)

# columns = [
#         TableColumn(field="text", title="errors"),
#     ]
# data_table = DataTable(source=source, columns=columns, width=400, height=280)


# top_words = cluster_words[cluster_num]
# top_words = pd.DataFrame(top_words).reset_index()
# top_words.columns = ['index','top_words']
# source2 = ColumnDataSource(top_words)

# columns2 = [
#         TableColumn(field="top_words", title="top words"),
#     ]
# data_table2 = DataTable(source=source2, columns=columns2, width=400, height=280)

# grid = gridplot([[p, None], [data_table, data_table2]])
# #show(row(p, data_table, data_table2))
# show(grid)

In [ ]:
cluster_words[cluster_num]

# Test